In [1]:
# import all the needed stuff:

import time
import sys
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt

import easygui  # popup windows with buttons made easy
import mne  # EEGLAB for python
from IPython.display import clear_output  # to clear the cell output during while loop
import re  # regular expressions
import pickle  # to save/load data
import dynarray  # a growing numpy array

import logging
logging.basicConfig(level=logging.ERROR)

sys.path.append("../../mushu")  # driver for the amps
sys.path.append("../../mushu/libmushu")
import libmushu

sys.path.append("../../nftools")  # handy stuff needed for NF
from nftools.loopcontrol import LoopState
from nftools.analysis import convert_alld_allm_to_mne
from nftools.analysis import select_part_from_mne_dataset
from nftools.analysis import plot_compare_two_spectra


sys.path.append("../../wyrm")  # real-time data analysis
from wyrm.types import RingBuffer
from wyrm.types import BlockBuffer
from wyrm import io
from wyrm import processing as proc
from wyrm import signaltracking
from wyrm import filtering

import scipy
from scipy import signal

from collections import deque  # a FILO list useful for plotting!

In [2]:
# so grab some data for replay:

# fn=easygui.fileopenbox(default='/home/johan/nf/')
# print(fn)

fn='/home/johan/nf/example-data.set'
raw_fromfile = mne.io.read_raw_eeglab(fn)
montage=mne.channels.read_montage('standard_1005', ch_names=raw_fromfile.ch_names)  # always use MNE definitions
raw_fromfile.set_montage(montage)


# properties of the recording, and what we're doing here:
fs = raw_fromfile.info['sfreq']
nbchan = raw_fromfile.info['nchan']-1

updateTime = 0.1  # run some kind of calculation every X seconds
buffSize = 1.0  # run calculation on last X seconds of data

The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
Events like the following will be dropped entirely: ['boundary', 'Impedance'], 2 in total
2/2561 event codes could not be mapped to integers. Use the 'event_id' parameter to map such events manually.
1 events will be dropped because they occur on the same time sample as another event. `mne.io.Raw` objects store events on an event channel, which cannot represent two events on the same sample. You can extract the original event structure using `mne.io.eeglab.read_events_eeglab`. Then, you can e.g. subset the extracted events for constructing epochs.
Data will be preloaded. preload=False or a string preload is not supported when the data is stored in the .set file


<ipython-input-2-3d41b03a8892>:7: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_fromfile = mne.io.read_raw_eeglab(fn)
<ipython-input-2-3d41b03a8892>:7: RuntimeWarning: Events like the following will be dropped entirely: ['boundary', 'Impedance'], 2 in total
  raw_fromfile = mne.io.read_raw_eeglab(fn)
<ipython-input-2-3d41b03a8892>:7: RuntimeWarning: 2/2561 event codes could not be mapped to integers. Use the 'event_id' parameter to map such events manually.
  raw_fromfile = mne.io.read_raw_eeglab(fn)
<ipython-input-2-3d41b03a8892>:7: RuntimeWarning: 1 events will be dropped because they occur on the same time sample as another event. `mne.io.Raw` objects store events on an event channel, which cannot represent two events on the same sample. You can extract the original event structure using `mne.io.eeglab.read_events_eeglab`. Then, you can e.g. subset the extracted events for constr

In [3]:
# prepare data for replay (warning: need probably a lot of memory)

mul_factor = 1.0
if 1e-6 in [raw_fromfile.info['chs'][0]['cal'], raw_fromfile.info['chs'][0]['range']]:
    mul_factor = 1.0 / 1e-6

seed_d=raw_fromfile[:-1,:][0] * mul_factor  # scale the data to seed (so no 1e-6 stuff in the replayed data)
seed_d=np.array(seed_d.transpose())
seed_ch=raw_fromfile.ch_names[0:-1]
seed_fs=raw_fromfile.info['sfreq']

# prepare for replay; markers:
seed_mdata=np.transpose(raw_fromfile[-1,:][0])
seed_m=[[i / raw_fromfile.info['sfreq'] * 1000, int(m[0])] for i, m in enumerate(seed_mdata) if m > 0] 

In [4]:
amp = libmushu.get_amp('replayamp')
amp.configure(seed_d, seed_m, seed_ch, seed_fs, realtime=True, blocksize_samples=10)

In [5]:
# for collecting the data again after replay:
alld=dynarray.DynamicArray((None, 2))     # data
allm=[]     # markers
sfreq = amp.get_sampling_frequency()  # sampling frequency
ch_names=amp.get_channels()  # channel names

markTime=time.time()


rb = RingBuffer(buffSize * 1000)  # the buffer containing the last X seconds of data - declared in MILISECONDS
totalTime = seed_d.shape[0]/raw_fromfile.info['sfreq']

## The following should likely be put into something more convenient like a class, or using wyrm's functionality

In [6]:
%matplotlib qt5  
plt.ion()  # enable widget plots & interactive plots

time_in_plot=10.0
sy1=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))  # for plotting - the FILO list
sy2=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))  # for plotting - the FILO list

channel_to_plot=0
sx = np.linspace(0, time_in_plot, round(fs * time_in_plot))

featuresy1 = deque(np.zeros(round(1/updateTime * time_in_plot)), round(1/updateTime * time_in_plot))
featuresx = np.linspace(0, time_in_plot, round(1/updateTime * time_in_plot))


In [7]:
# from wyrm import signaltracking
eegfilter = filtering.bwBPF(12,15)
emgfilter = filtering.bwHPF(50)
# smoother = filtering.bwExponentialSmoother(0.998)
# smoother = filtering.bwLPF(2,order=2)
smoother_eeg = filtering.bwSimpleAverager(300,2)
smoother_emg = filtering.bwSimpleAverager(300,2)

track_for_eeg_stimuli = signaltracking.above_thr(80, 0.2)
track_for_emg_stimuli = signaltracking.above_thr(100, 0.15)

bwBPF: {'f_low': 12, 'f_high': 15, 'order': 3}
bwHPF: {'f': 50, 'order': 3}
bwSimpleAverager: {'taps': 300, 'f': 2}
bwSimpleAverager: {'taps': 300, 'f': 2}
thr: 80.00, dur: 0.20
thr: 100.00, dur: 0.15


### until here

In [8]:
amp.start()

In [ ]:
fig=plt.figure()  # plotting...
th=fig.suptitle('')
ah1=fig.add_subplot(111)
# ah2=fig.add_subplot(122)
l1, = ah1.plot(sx, sy1)
l2, = ah1.plot(sx, sy2)

# featurefig = plt.figure()
# featureth=featurefig.suptitle('')
# featureah=featurefig.add_subplot(111)
# featurel1, = featureah.plot(featuresx, featuresy1)



# l=LoopState(); l.start()
markeroffset = 0  # needed to store all data in one big mat/vector
t0=time.time()
curTime=time.time()
st=''
while curTime - t0 < totalTime:  # l.get_state() != 'Stop':
   
    
    # keep track of time:
    curTime = time.time()
    
    # this is where you get the data
    data, marker = amp.get_data()
    
    if data.shape[0] > 0:  # this is crucual for remembering filter state.
        
        
        cnt = io.convert_mushu_data(data, marker, sfreq, ch_names)


        
        eeg_cnt = proc.select_channels(cnt, ['C3','C4'])
        f_eeg_cnt = eegfilter.apply(eeg_cnt)  # so filter it according to specified above
        af_eeg_cnt = proc.absolute(f_eeg_cnt)
        saf_eeg_cnt = smoother_eeg.apply(af_eeg_cnt)
        track_for_eeg_stimuli.check_rising(saf_eeg_cnt)

        
        # do the EMG:
        emg_cnt = proc.select_channels(cnt, ['O1','O2'])
        f_emg_cnt = emgfilter.apply(emg_cnt)  # so filter it according to specified above
        af_emg_cnt = proc.absolute(f_emg_cnt)
        saf_emg_cnt = smoother_emg.apply(af_emg_cnt)
        track_for_emg_stimuli.check_rising(af_emg_cnt)
        
        # f_cnt = f_eeg_cnt
        
        
        alld.extend(eeg_cnt.data)
        for m in marker:
            allm.append([m[0] + markeroffset, m[1]])
        markeroffset += cnt.data.shape[0] / float(sfreq) * 1000.
        

        # plotting...
        sy1.extend(af_eeg_cnt.data[:,0])  # to visualize/plot -- s1 and s2 are deque's
        sy2.extend(saf_eeg_cnt.data[:,0])
        l1.set_ydata(sy1)
        l2.set_ydata(sy2)
        msy1=np.mean(sy1)
        msy2=np.mean(sy2)
        ah1.set_ylim(-20+msy1, 20+msy1)
        # ah2.set_ylim(-20+msy2, 20+msy2)

        fig.canvas.draw()
        fig.canvas.flush_events()

        
        

        # append to ringbuffer, so we can calculate features later on on the last N secs/samples of data.
        rb.append(f_eeg_cnt)



        # do the following every 0.1 msec - with with the ringbuffer:
        if curTime - markTime > updateTime:
            # do Stuff

            markTime = curTime
            # 1) obtain last 1-second(s)
            # d = rb.get()

            # thomas does stuff here - in this example, we take channel 3 of the data and filter it
            # feature = np.log10(np.mean(abs(d.data[:,0]))) * 10
            # featuresy1.append(feature)


            # we send the value to BCI/STIM here - but not right now
            # bcinet.send_signal(bcixml.BciSignal({'nfsignal': signalToSend},None, bcixml.CONTROL_SIGNAL))
            # featureMax = 0
            # featureMin = -6.0
            # featureScaling = 1/abs(featureMax - featureMin)
            # featureOffset = (featureMax + featureMin) / 2
            # signalToSend = featureScaling * (feature - featureOffset)

            # plot of the feature in a separate figure, to keep track:
            # featurel1.set_ydata(featuresy1) # plotting the feature stuff
            # featuremsy1=np.mean(featuresy1)
            # featureah.set_ylim(-10+featuremsy1, 10+featuremsy1)
            # featurefig.canvas.draw()
            # featurefig.canvas.flush_events()
            

            # clear_output(wait=True)  # write some logging information here
            # clear_output clear the output of the cell, but if you do that you also remove the figures, it seems
            # so don't do it!
            str1 = 'Playing Back - time = %f' % (curTime - t0)
            str2 = 'Length Markers: %d' % len(allm)
            str3 = '%d, %d' % data.shape
            # str4 = 'Feature Value: %f' % feature
            # str5 = 'Scaled Signal for NF: %f' % signalToSend
            #print(str1 + '\n' + str2 + '\n' + str3 + '\n' + str4 + '\n' + str5)
            
            # print('Length Markers: %d' % len(allm))
            # print(data.shape)
            th.set_text(str1 + '\n' + str2 + '\n' +str3)
            # featureth.set_text(str4 + '\n' + str5)





/home/johan/.conda/envs/rt/lib/python3.6/site-packages/scipy/signal/signaltools.py:1341: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out_full[ind] += zi
/home/johan/.conda/envs/rt/lib/python3.6/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]
/home/johan/.conda/envs/rt/lib/python3.6/site-packages/scipy/signal/signaltools.py:1350: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future

In [17]:
cnt.data.copy()

array([[ -8698.31054688, -17333.484375  ,  -6285.11230469, ...,
         -8208.1015625 ,  -3089.72509766,  -3513.5246582 ],
       [ -8693.24707031, -17333.54882812,  -6284.4453125 , ...,
         -8211.38671875,  -3093.75341797,  -3515.08178711],
       [ -8686.70019531, -17331.48828125,  -6282.11474609, ...,
         -8215.05859375,  -3098.890625  ,  -3520.3815918 ],
       ...,
       [ -8672.5       , -17291.49414062,  -6306.49316406, ...,
         -8228.93457031,  -3129.15454102,  -3543.21875   ],
       [ -8666.33691406, -17284.7421875 ,  -6305.91259766, ...,
         -8229.21875   ,  -3129.51953125,  -3545.85766602],
       [ -8664.0234375 , -17294.31835937,  -6302.55273437, ...,
         -8228.78710938,  -3124.10180664,  -3542.80224609]])

In [15]:
plt.figure;plt.plot(af_eeg_cnt.data)

In [11]:
af_eeg_cnt

In [14]:
plt.figure()
plt.plot(smoother.b)

In [15]:
eeg_cnt.data.shape

(500, 2)

In [35]:
plt.figure()
plt.plot(proc.absolute(f_eeg_cnt).data)

In [16]:
eegfilter.b

array([ 8.21609743e-07,  0.00000000e+00, -2.46482923e-06,  0.00000000e+00,
        2.46482923e-06,  0.00000000e+00, -8.21609743e-07])

In [2]:
smoother.zi

NameError: name 'smoother' is not defined

In [10]:
alld.shrink_to_fit()

In [22]:
v=alld[:,0]

In [23]:
plt.figure()

<Figure size 640x480 with 0 Axes>

In [24]:
plt.plot(smoother.b)

In [25]:
zi = signal.lfilter_zi(smoother.b, 1.0)

In [26]:
plt.figure();plt.plot(smoother.zi)

In [14]:
fv=signal.lfilter(smoother.b, 1.0, v, axis=0)

/home/johan/.conda/envs/rt/lib/python3.6/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]


In [15]:
plt.plot(fv)

In [17]:
plt.figure()
plt.plot(smoother.zi)

In [18]:
zi=signal.lfiltic(smoother.b, 1.0, y=np.random.random((10,4)), x=np.random.random((10,4)))

ValueError: all the input arrays must have same number of dimensions

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from scipy import signal

In [ ]:
%matplotlib qt

In [31]:
for i, v in enumerate(cnt.data.copy()):
    print(v)
    print(i)

[ -8949.09570312 -17578.02539062  -6526.35742187 -10147.28222656
  -1737.39587402  -4909.40820312  -3469.99487305 -10258.97363281
  -6125.62939453  -2623.55981445  -8521.0625      -3356.83251953
  -3753.22412109]
0
[ -8956.48925781 -17584.65820312  -6528.56982422 -10145.55859375
  -1737.18457031  -4907.89794922  -3468.47094727 -10258.84082031
  -6125.64453125  -2621.71728516  -8518.42480469  -3350.35400391
  -3753.09130859]
1
[ -8970.42675781 -17585.85546875  -6532.98193359 -10149.93945312
  -1740.54101562  -4908.6640625   -3469.00561523 -10258.67578125
  -6124.03662109  -2615.64990234  -8509.91699219  -3336.71240234
  -3739.31835938]
2
[ -8974.49511719 -17583.96484375  -6533.07324219 -10150.15429688
  -1742.06689453  -4907.86572266  -3468.078125   -10259.75
  -6124.39697266  -2611.15576172  -8521.63574219  -3331.03588867
  -3743.26269531]
3
[ -8966.19726562 -17577.76171875  -6527.78759766 -10144.63964844
  -1740.65393066  -4906.92041016  -3467.98681641 -10263.26757812
  -6124.01269531

In [ ]:
b=signal.firwin(300,2*3/1000)

In [ ]:
zi = signal.lfilter_zi(b, 1)

In [ ]:
plt.plot(b)

In [ ]:
plt.show()

In [ ]:
d.units

In [ ]:
f_cnt.data.shape

In [ ]:
import signal

In [ ]:
from scipy import signal

In [ ]:
from scipy.signal import lfilter_zi